In [8]:
# Standard libraries
import datetime
import math
import pathlib
import sqlite3
import sys

# Installed libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import spiceypy
import tqdm

# Append to root directory of this repository
sys.path.append("../")

# Auxiliary module that contains the apparent magnitude
from auxiliary import photometry

In [2]:
# Accessing the NEO database
database_dir = pathlib.Path("../databases/neos/")
database_file = pathlib.Path("neodys.db")
database_filepath = database_dir / database_file

# Establish a connection to the database and set a cursor
neodys_db_con = sqlite3.connect(database_filepath)
neodys_db_cur = neodys_db_con.cursor()

In [3]:
# Get all information from the DB. Since the DB is rather small, this won't cause any issues!
neo_df = pd.read_sql("SELECT * FROM main", neodys_db_con)

# Close the database.
neodys_db_con.close()

In [4]:
# Load SPICE kernels
spiceypy.furnsh("../kernels/spk/de432s.bsp")
spiceypy.furnsh("../kernels/lsk/naif0012.tls")
spiceypy.furnsh("../kernels/pck/gm_de431.tpc")

# Get the G*M value of the Sun
_, gm_sun_pre = spiceypy.bodvcd(bodyid=10, item='GM', maxn=1)
gm_sun = gm_sun_pre[0]

In [5]:
# For our computations we need to convert some values from AU to km and from deg to rad
neo_df.loc[:, "Perihel_km"] = neo_df["Perihel_AU"].apply(lambda x: spiceypy.convrt(x, "AU", "km"))
neo_df.loc[:, "Incl_rad"] = neo_df["Incl_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "LongAscNode_rad"] = neo_df["LongAscNode_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "ArgP_rad"] = neo_df["ArgP_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "MeanAnom_rad"] = neo_df["MeanAnom_deg"].apply(lambda x: math.radians(x))
neo_df.loc[:, "Epoch_JD"] = neo_df["Epoch_MJD"].apply(lambda x: x + 2400000.5)
neo_df.loc[:, "Epoch_et"] = neo_df["Epoch_JD"].apply(lambda x: spiceypy.utc2et(str(x) + " JD"))

In [6]:
init_time_utc = "2022-01-01T00:00:00"
init_time_et = spiceypy.utc2et(init_time_utc)

opp_range = 45.0

detected_neo_df = pd.DataFrame([])

In [10]:
while len(neo_df) > 0:
    for time_step in tqdm.tqdm(np.arange(0, 8760 * 10, 1)):

        init_time_et += time_step
        
        sun2earth_position_vec = spiceypy.spkgps(targ=399,
                                             et=init_time_et,
                                             ref="ECLIPJ2000",
                                             obs=10)[0]
        earth2sun_position_vec = -1.0 * sun2earth_position_vec

        neo_df.loc[:, "et_of_detection"] = init_time_et


        # Compute the position vector of each NEO as seen from the Sun
        neo_df.loc[:, "sun2neo_position_vec"] = \
            neo_df.apply(lambda x: spiceypy.conics(elts=[x["Perihel_km"],
                                                         x["Ecc_"],
                                                         x["Incl_rad"],
                                                         x["LongAscNode_rad"],
                                                         x["ArgP_rad"],
                                                         x["MeanAnom_rad"],
                                                         x["Epoch_et"],
                                                         gm_sun],
                                                 et=init_time_et)[:3],
                         axis=1)

        # To compute the apparent magnitude we need to re-compute the positional vectors and convert it to
        # AU
        neo_df.loc[:, "neo2earth_position_vec"] = \
            neo_df["sun2neo_position_vec"].apply(lambda x: sun2earth_position_vec - x)

        neo_df.loc[:, "neo2sun_position_vec"] = \
            neo_df["sun2neo_position_vec"].apply(lambda x: -1.0 * x)

        neo_df.loc[:, "neo2earth_position_vec_AU"] = \
            neo_df["neo2earth_position_vec"].apply(lambda x: [spiceypy.convrt(k, "km", "AU") for k in x])
        neo_df.loc[:, "neo2sun_position_vec_AU"] = \
            neo_df["neo2sun_position_vec"].apply(lambda x: [spiceypy.convrt(k, "km", "AU") for k in x])

        neo_df.loc[:, "earth2neo_position_vec_AU"] = \
            neo_df["neo2earth_position_vec_AU"].apply(lambda x: -1.0 * np.array(x))
        
        # Compute the apparent magnitude of each NEO for today!
        neo_df.loc[:, "app_mag"] = \
            neo_df.apply(lambda x: photometry.hg_app_mag(abs_mag=x["AbsMag_"],
                                                         vec_obj2obs=x["neo2earth_position_vec_AU"],
                                                         vec_obj2ill=x["neo2sun_position_vec_AU"],
                                                         slope_g=x["SlopeParamG_"]), axis=1)

        neo_df.loc[:, "ang_dist_neo2opp_deg"] = \
            neo_df["earth2neo_position_vec_AU"].apply(lambda x:
                                                      np.degrees(spiceypy.vsep(x, -1.0*earth2sun_position_vec)))
        
        rows = neo_df.loc[(neo_df["app_mag"] <= 26.0) & (neo_df["ang_dist_neo2opp_deg"] <= opp_range), :]
        detected_neo_df.append(rows, ignore_index=True)
        neo_df.drop(rows.index, inplace=True)

  0%|                                                          | 7/87600 [00:27<97:13:32,  4.00s/it]


KeyboardInterrupt: 